# Data Download

In [ ]:
!wget https://data.mendeley.com/public-files/datasets/fwcj7stb8r/files/fd8717c4-0d65-4c80-a76c-3b61cb68e80d/file_downloaded

In [ ]:
!7za x file_downloaded

In [ ]:
!rm -r sample_data
!rm file_downloaded
!mv 'Rice Leaf Disease Images' data
!ls

# Backend Part

In [ ]:
!pip install tensorflow

**Import Libraries**

In [5]:
import tensorflow as tf
import os
from tensorflow.keras import layers, datasets, models
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn import utils

In [6]:
CATEGORIES = ["Bacterialblight" , "Blast" , "Brownspot", "Tungro"]
IMG_SIZE = 200

In [7]:
X = list()
y = list()

def createTrainingData():
  for category in CATEGORIES:
    path = os.path.join('/content/data',category)
    category_index = CATEGORIES.index(category)
    for img in os.listdir(path):
      img_arr = cv2.imread(os.path.join(path, img))
      new_arr = cv2.resize(img_arr , (IMG_SIZE, IMG_SIZE))
      X.append(new_arr)
      y.append(category_index)

createTrainingData()
X = np.array(X)
y = np.array(y)   

X, y = utils.shuffle(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
cnn = models.Sequential([
  layers.Conv2D(32 ,(3,3), activation='relu' , input_shape=(IMG_SIZE,IMG_SIZE,3)),
  layers.MaxPooling2D((2,2), strides = 2),
  layers.Conv2D(32 , (3,3), activation='relu'),
  layers.MaxPooling2D((2,2), strides = 2),
  layers.Dropout(0.5),
  layers.Flatten(),
  layers.Dense(1000, activation='relu'),
  layers.Dense(4 , activation='softmax')
])
cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
cnn.fit(X_train, y_train, epochs=20)
cnn.evaluate(X_test, y_test)

In [10]:
def predict(filename):
  path = os.path.join('/content/static/upload', filename)
  img = cv2.imread(path)
  x = img.shape[1]
  y = img.shape[0]
  x1 = (x - IMG_SIZE)//2
  y1 = (y - IMG_SIZE)//2
  x2 = x - x1
  y2 = y - y1
  img = img[y1:y2, x1:x2]
  img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
  imgl = [img]
  imgl = np.array(imgl)
  
  index = np.argmax(cnn.predict(imgl))
  return CATEGORIES[index]

# File Setup for frontend

In [ ]:
!rm -r static templates app.py
!git clone https://github.com/KalWardinX/crop_disease_detection.git
!mv crop_disease_detection/code/static ./static
!mv crop_disease_detection/code/templates ./templates
!mv crop_disease_detection/code/app.py ./app.py
!rm -r crop_disease_detection

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xf ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 26Xss91r59HCCYKbBsKJsbAhRK7_6QgLMnQQtK4kBhtp6NtMg

# Frontend Part

In [ ]:
!pip install flask_ngrok

**main.py**

In [14]:
from flask_ngrok import run_with_ngrok
from flask import Flask , render_template,request, flash, redirect, url_for
import os
from werkzeug.utils import secure_filename

In [ ]:
UPLOAD_FOLDER = 'static/upload'
ALLOWED_EXTENSIONS = {'png', 'jpg' ,'jpeg'}

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.secret_key = 'secret key'
app.config['SESSION_TYPE'] = 'filesystem'
run_with_ngrok(app)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/',methods=['GET'])
def index():
    for i in os.listdir(UPLOAD_FOLDER):
        os.remove(UPLOAD_FOLDER+'/'+i)
    return render_template('index.htm', title='crop leaf disease')

@app.route('/',methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        flash("no file part")
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash("no imiage selected for uploading")
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        disease = predict(filename)
        return render_template('index.htm', filename=filename, disease=disease)
    else:
        flash("not allowed file")
        return redirect(request.url)

@app.route('/upload/<filename>')
def display_image(filename):
    return redirect(url_for('static', filename='upload/'+filename),code=301)

if __name__ == "__main__":
    app.run()
